# Biostat 257 HW2
### Caesar Z. Li
### UID: 704135662

## Q1. (Optional, 30 bonus pts) Derivatives
Answer: show proof:

\begin{eqnarray*}
\ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) = - \frac{n_i}{2} \log (2\pi) - \frac{1}{2} \log \det \boldsymbol{\Omega}_i - \frac{1}{2} (\mathbf{y} - \mathbf{X}_i \boldsymbol{\beta})^T \boldsymbol{\Omega}_i^{-1} (\mathbf{y} - \mathbf{X}_i \boldsymbol{\beta})
\end{eqnarray*}

where
\begin{eqnarray*}
 \boldsymbol{\Omega}_i = \sigma^2 \mathbf{I}_{n_i} + \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T = \sigma^2 \mathbf{I}_{n_i} + \mathbf{Z}_i \mathbf{L} \mathbf{L}^T \mathbf{Z}_i^T
\end{eqnarray*}

Simpler way to find $\nabla_{\boldsymbol{\beta}} \ell_i$: use chain rule on $\mathbf{r}_i$
\begin{eqnarray*}
\nabla_{\boldsymbol{\beta}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) = \mathbf{D}_{\mathbf{r}_i} \partial \mathbf{r}^{T}_i \mathbf{\Omega}^{-1}\mathbf{r}_i  \cdot \mathbf{D}_{\boldsymbol{\beta}} \mathbf{r}_i \\
= - (\boldsymbol{\Omega}^{-1}\mathbf{r}_i)^T (-\mathbf{X}_i) = \mathbf{r}^T_i\boldsymbol{\Omega}^{-1}\mathbf{X}_i = \mathbf{X}_i\boldsymbol{\Omega}^{-1}\mathbf{r}^T_i
\end{eqnarray*}

Second, to find $\nabla_{\sigma^2} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2)$, first find,

\begin{eqnarray*}
\mathbf{D}_{\boldsymbol{\sigma}^2} \boldsymbol{\Omega} = vec\mathbf{I}
\end{eqnarray*}

And then,
$$
\mathbf{D}_{\boldsymbol{\Omega}} \ell_i = -\frac{1}{2\det(\boldsymbol{\Omega})} \det(\boldsymbol{\Omega}) (\boldsymbol{\Omega}^T)^{-1} - \mathbf{D}_{\boldsymbol{\Omega}}\frac{1}{2}tr(\mathbf{r}^T_i\boldsymbol{\Omega}^{-1}\mathbf{r}_i)  \\
= -\frac{1}{2}(\boldsymbol{\Omega}^T)^{-1} - \mathbf{D}_{\boldsymbol{\Omega}}\frac{1}{2}tr(\mathbf{r}_i\mathbf{r}^T_i\boldsymbol{\Omega}^{-1}) \\
= -\frac{1}{2}\boldsymbol{\Omega}^{-1} + \frac{1}{2}\boldsymbol{\Omega}^{-1}\mathbf{r}_i\mathbf{r}^T_i\boldsymbol{\Omega}^{-1}
$$

Use the chain rule,
$$
\nabla_{\sigma^2} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) = \\
\mathbf{D}_{\boldsymbol{\Omega}} \ell_i \cdot \mathbf{D}_{\boldsymbol{\sigma}^2} \boldsymbol{\Omega} \\
= -\frac{1}{2}vec(\boldsymbol{\Omega}^{-1})^Tvec\mathbf{I} + \frac{1}{2}vec(\boldsymbol{\Omega}^{-1}\mathbf{r}_i\mathbf{r}^T_i\boldsymbol{\Omega}^{-1})^T vec\mathbf{I} \\
=-\frac{1}{2}tr(\boldsymbol{\Omega}^T)^{-1} + \frac{1}{2}tr(\boldsymbol{\Omega}^{-1}\mathbf{r}_i\mathbf{r}^T_i\boldsymbol{\Omega}^{-1}) \\
=-\frac{1}{2}tr(\boldsymbol{\Omega}^T)^{-1} + \frac{1}{2}tr(\boldsymbol{\Omega}^{-1}\boldsymbol{\Omega}^{-1}\mathbf{r}_i\mathbf{r}^T_i)\\
=-\frac{1}{2}tr(\boldsymbol{\Omega}^T)^{-1} + \frac{1}{2}tr(\mathbf{r}^T_i\boldsymbol{\Omega}^{-1}\boldsymbol{\Omega}^{-1}\mathbf{r}_i)\\
= -\frac{1}{2}tr(\boldsymbol{\Omega}^T)^{-1} + \frac{1}{2}\mathbf{r}^T_i\boldsymbol{\Omega}^{-2}\mathbf{r}_i
$$

Lastly, to find $\mathbf{D}_{\boldsymbol{L}} \ell_i$, first get,
$$
\mathbf{D}_{\mathbf{L}} \boldsymbol{\Omega} = \mathbf{D}_{\mathbf{L}}  (\sigma^2 \mathbf{I}_{n_i} + \mathbf{Z}_i \mathbf{L} \mathbf{L}^T \mathbf{Z}_i^T) \\
= \mathbf{Z}_i \mathbf{L} \otimes \mathbf{Z}_i + (\mathbf{Z}_i \otimes \mathbf{Z}_i \mathbf{L}) \mathbf{K}_{qq} \\
= \mathbf{Z}_i \mathbf{L} \otimes \mathbf{Z}_i + \mathbf{K}_{qq} (\mathbf{Z}_i \mathbf{L} \otimes \mathbf{Z}_i). \\
$$
Then,
$$
\mathbf{D}_{\boldsymbol{L}} \ell_i = \mathbf{D}_{\boldsymbol{\Omega}}\ell_i \cdot \mathbf{D}_{\mathbf{L}}\boldsymbol{\Omega} \\
= -\frac{1}{2}vec(\boldsymbol{\Omega}^{-1})^T(\mathbf{Z}_i \mathbf{L} \otimes \mathbf{Z}_i) - \frac{1}{2}(\mathbf{K}_{qq}vec(\boldsymbol{\Omega}^{-1}))^T(\mathbf{Z}_i \mathbf{L} \otimes \mathbf{Z}_i) + \frac{1}{2}vec(\boldsymbol{\Omega}^{-1}\mathbf{r}_i\mathbf{r}^T_i\boldsymbol{\Omega}^{-1})^T(\mathbf{Z}_i \mathbf{L} \otimes \mathbf{Z}_i) + \frac{1}{2}(\mathbf{K}_{qq}vec(\boldsymbol{\Omega}^{-1}\mathbf{r}_i\mathbf{r}^T_i\boldsymbol{\Omega}^{-1}))^T(\mathbf{Z}_i \mathbf{L} \otimes \mathbf{Z}_i) \\
= -\frac{1}{2}((\mathbf{L}^T\mathbf{Z}^T_i  \otimes \mathbf{Z}^T_i)vec(\boldsymbol{\Omega}^{-1}))^T - -\frac{1}{2}((\mathbf{L}^T\mathbf{Z}^T_i  \otimes \mathbf{Z}^T_i)vec(\boldsymbol{\Omega}^{-1}))^T  + \frac{1}{2}((\mathbf{L}^T\mathbf{Z}^T_i  \otimes \mathbf{Z}^T_i)vec(\boldsymbol{\Omega}^{-1}\mathbf{r}_i\mathbf{r}^T_i\boldsymbol{\Omega}^{-1}))^T + \frac{1}{2}((\mathbf{L}^T\mathbf{Z}^T_i  \otimes \mathbf{Z}^T_i)vec(\boldsymbol{\Omega}^{-1}\mathbf{r}_i\mathbf{r}^T_i\boldsymbol{\Omega}^{-1}))^T \\
-vec(\mathbf{Z}^T_i\boldsymbol{\Omega}^{-1}\mathbf{Z}_i\mathbf{L})^T + vec(\mathbf{Z}^T_i\boldsymbol{\Omega}^{-1}\mathbf{r}_i\mathbf{r}^T_i\boldsymbol{\Omega}^{-1}\mathbf{Z}_i\mathbf{L})^T \\
= -\mathbf{Z}^T_i\boldsymbol{\Omega}^{-1}\mathbf{Z}_i\mathbf{L} + \mathbf{Z}^T_i\boldsymbol{\Omega}^{-1}\mathbf{r}_i\mathbf{r}^T_i\boldsymbol{\Omega}^{-1}\mathbf{Z}_i\mathbf{L}
$$

In [1]:
# load necessary packages; make sure install them first
using BenchmarkTools, CSV, DataFrames, DelimitedFiles, Distributions
using Ipopt, LinearAlgebra, MathProgBase, MixedModels, NLopt
using Random, RCall, Revise

## Q2. (20 pts) Objective and gradient evaluator for a single datum

In [2]:
# define a type that holds an LMM datum
struct LmmObs{T <: AbstractFloat}
    # data
    y          :: Vector{T}
    X          :: Matrix{T}
    Z          :: Matrix{T}
    # arrays for holding gradient
    ∇β         :: Vector{T}
    ∇σ²        :: Vector{T}
    ∇Σ         :: Matrix{T}    
    # working arrays
    # TODO: whatever intermediate arrays you may want to pre-allocate
    yty        :: T
    xty        :: Vector{T}
    zty        :: Vector{T}
    storage_p  :: Vector{T}
    storage_q  :: Vector{T}
    storage_q2 :: Vector{T}
    storage_q3 :: Vector{T}
    xtx        :: Matrix{T}
    ztx        :: Matrix{T}
    ztz        :: Matrix{T}
    storage_qq :: Matrix{T}
    storage_qq2 :: Matrix{T}
    storage_qq3 :: Matrix{T}
end

In [3]:

"""
    LmmObs(y::Vector, X::Matrix, Z::Matrix)

Create an LMM datum of type `LmmObs`.
"""
function LmmObs(
        y::Vector{T}, 
        X::Matrix{T}, 
        Z::Matrix{T}
    ) where T <: AbstractFloat
    n, p, q    = size(X, 1), size(X, 2), size(Z, 2)    
    ∇β         = Vector{T}(undef, p)
    ∇σ²        = Vector{T}(undef, 1)
    ∇Σ         = Matrix{T}(undef, q, q)    
    yty        = abs2(norm(y))
    xty        = transpose(X) * y
    zty        = transpose(Z) * y    
    storage_p  = Vector{T}(undef, p)
    storage_q  = Vector{T}(undef, q)
    storage_q2 = Vector{T}(undef, q)
    storage_q3 = Vector{T}(undef, q)
    xtx        = transpose(X) * X
    ztx        = transpose(Z) * X
    ztz        = transpose(Z) * Z
    storage_qq = similar(ztz)
    storage_qq2 = similar(ztz)
    storage_qq3 = similar(ztz)
    LmmObs(y, X, Z, ∇β, ∇σ², ∇Σ, 
        yty, xty, zty, storage_p, storage_q, storage_q2, storage_q3, 
        xtx, ztx, ztz, storage_qq, storage_qq2, storage_qq3)
end

"""
    logl!(obs::LmmObs, β, L, σ², needgrad=false)

Evaluate the log-likelihood of a single LMM datum at parameter values `β`, `L`, 
and `σ²`. If `needgrad==true`, then `obs.∇β`, `obs.∇Σ`, and `obs.σ² are filled 
with the corresponding gradient.
"""
function logl!(
        obs      :: LmmObs{T}, 
        β        :: Vector{T}, 
        L        :: Matrix{T}, 
        σ²       :: T,
        needgrad :: Bool = true
    ) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)
    ####################
    # Evaluate objective
    ####################    
    # form the q-by-q matrix: M = σ² * I + Lt Zt Z L
    copy!(obs.storage_qq, obs.ztz)
    BLAS.trmm!('L', 'L', 'T', 'N', T(1), L, obs.storage_qq) # O(q^3)
    BLAS.trmm!('R', 'L', 'N', 'N', T(1), L, obs.storage_qq) # O(q^3)
    @inbounds for j in 1:q
        obs.storage_qq[j, j] += σ²
    end
    # cholesky on M = σ² * I + Lt Zt Z L
    LAPACK.potrf!('U', obs.storage_qq) # O(q^3)
    # storage_q = (Mchol.U') \ (Lt * (Zt * res))
    BLAS.gemv!('N', T(-1), obs.ztx, β, T(1), copy!(obs.storage_q, obs.zty)) # O(pq)
    BLAS.trmv!('L', 'T', 'N', L, obs.storage_q)    # O(q^2)
    BLAS.trsv!('U', 'T', 'N', obs.storage_qq, obs.storage_q) # O(q^3)
    # l2 norm of residual vector
    copy!(obs.storage_p, obs.xty)
    rtr  = obs.yty +
        dot(β, BLAS.gemv!('N', T(1), obs.xtx, β, T(-2), obs.storage_p))
    # assemble pieces
    logl::T = n * log(2π) + (n - q) * log(σ²) # constant term
    @inbounds for j in 1:q
        logl += 2log(obs.storage_qq[j, j])
    end
    qf    = abs2(norm(obs.storage_q)) # quadratic form term
    logl += (rtr - qf) / σ² 
    logl /= -2
    ###################
    # Evaluate gradient
    ###################    
    if needgrad
        # TODO: fill ∇β, ∇L, ∇σ² by gradients
        # form ∇β first:
        # fill in ∇β with Xt(y - Xβ) = xty - XtXβ
        copy!(obs.∇β, obs.xty)
        BLAS.gemv!('N', T(-1), obs.xtx, β, T(1), obs.∇β)
        # storage_q2 = (Mchol.U) \ ( (Mchol.U') \ (Lt * (Zt * res)) )
        copy!(obs.storage_q2, obs.storage_q)
        BLAS.trsv!('U', 'N', 'N', obs.storage_qq, obs.storage_q2)
        BLAS.trmv!('L', 'N', 'N', L, obs.storage_q2) 
        # Put two pieces together, while makeing storage_q2 .= XtZ * storage_q2
        BLAS.gemv!('T', T(-1), obs.ztx, obs.storage_q2, T(1), obs.∇β)
        obs.∇β ./= σ²
        
        # form ∇σ²:
        # expand rtΩ⁻²r into rtr + ( rt * ZLM⁻¹ Lt ) * ZtZ * ( LM⁻¹LtZt * r ) +
        #  rt * ZLM⁻¹ LtZt * r 
        # Note the third term is already computed as 'qf' in earlier code
        # rtr is computed as well, so I need only compute the second term.
        BLAS.gemv!('N', T(1), obs.ztz, obs.storage_q2, T(0), obs.storage_q)
        obs.∇σ²[1] = dot(obs.storage_q2, obs.storage_q)
        obs.∇σ²[1] += rtr - 2qf
        obs.∇σ²[1] /= σ²
        # minus tr(Ω⁻¹)        
        copy!(obs.storage_qq3, obs.ztz)
        BLAS.trmm!('L', 'L', 'T', 'N', T(1), L, obs.storage_qq3)
        LAPACK.potrs!('U', obs.storage_qq, obs.storage_qq3)
        BLAS.trmm!('L', 'L', 'N', 'N', T(1), L, obs.storage_qq3)
        #BLAS.gemm!('N', 'N', T(1), L, obs.storage_qq3, T(0), obs.storage_qq2)
        @inbounds for j in 1:q
             obs.∇σ²[1] += obs.storage_qq3[j, j]
        end
        obs.∇σ²[1] -= n 
        obs.∇σ²[1] /= (2 * σ²)
        
        # form ∇Σ:
        # first part, get (-ZtZ + ZtZ * LM⁻¹Lt * ZtZ) 
        # or ZtZ (-I + LM⁻¹Lt * ZtZ)
        # put first half in obs.storage_qq2
        @inbounds for j in 1:q
             obs.storage_qq3[j, j] -= 1.0
        end
        #BLAS.trmm!('L', 'L', 'N', 'N', T(1), L, obs.storage_qq3)
        BLAS.gemm!('N', 'N', T(1), obs.ztz, obs.storage_qq3, T(0), obs.∇Σ)
        
        # second part, use Ω⁻¹r from earlier to get ZΩ⁻¹r 
        copy!(obs.storage_q, obs.zty)
        BLAS.gemv!('N', T(- 1), obs.ztx, β, T(1), obs.storage_q)
        # we have storage_q2 = L(Mchol.U) \ ( (Mchol.U') \ (Lt * (Zt * res)) )
        # Put two pieces together, only this time storage_q2 .= ZtZ * storage_q2
        BLAS.gemv!('N', T(- 1), obs.ztz, obs.storage_q2, T(1), obs.storage_q)
        # outer product this vector, then don't * L to get dSigma
        BLAS.gemm!('N', 'T', T(1 / σ²), obs.storage_q, obs.storage_q, T(1), obs.∇Σ)
        obs.∇Σ ./= σ²
        
        #sleep(1e-3) # pretend this step takes 1ms
    end    
    ###################
    # Return
    ###################        
    return logl    
end

logl!

It is a good idea to test correctness and efficiency of the single datum objective/gradient evaluator here. First generate the same data set as in HW2.

In [4]:
Random.seed!(257)
# dimension
n, p, q = 2000, 5, 3
# predictors
X  = [ones(n) randn(n, p - 1)]
Z  = [ones(n) randn(n, q - 1)]
# parameter values
β  = [2.0; -1.0; rand(p - 2)]
σ² = 1.5
Σ  = fill(0.1, q, q) + 0.9I # compound symmetry 
L  = Matrix(cholesky(Symmetric(Σ)).L)
# generate y
y  = X * β + Z * rand(MvNormal(Σ)) + sqrt(σ²) * randn(n)

# form the LmmObs object
obs = LmmObs(y, X, Z);

### 2.1  Correctness

In [36]:
@show logl = logl!(obs, β, L, σ², true)
@show obs.∇β
@show obs.∇σ²
@show obs.∇Σ;

logl = logl!(obs, β, L, σ², true) = -3247.4568580638256
obs.∇β = [-1.6309843232714532, -77.527515580418, -14.702372116010645, 6.978485518990293, -57.711823176822044]
obs.∇σ² = [-4.820377758258701]
obs.∇Σ = [1.6423791649320858 1.825024077223936 0.06127650043327201; 1.8250240772239479 0.11072391370464318 0.07213050869968186; 0.0612765004332986 0.07213050869966357 -1.0173748515301833]


You will lose all 20 points if following statement throws `AssertionError`.

In [11]:
@assert abs(logl - (-3247.4568580638247)) < 1e-8
@assert norm(obs.∇β - [-1.63098432327115, -77.52751558041871, -14.70237211601065, 
        6.978485518989568, -57.71182317682199]) < 1e-8
@assert norm(obs.∇Σ - [1.6423791649290531 1.82502407722348 0.06127650043330721; 
        1.82502407722348 0.1107239137055005 0.07213050869971993; 
        0.06127650043330721 0.07213050869971993 -1.0173748515299939]) < 1e-8
@assert abs(obs.∇σ²[1] - (-4.8203777582588145)) < 1e-8

### 2.2  Efficiency
Benchmark for evaluating objective only. This is what we did in HW2.

In [9]:
@benchmark logl!($obs, $β, $L, $σ², false)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.827 μs (0.00% GC)
  median time:      1.837 μs (0.00% GC)
  mean time:        1.850 μs (0.00% GC)
  maximum time:     43.715 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     10

In [27]:
bm_objgrad = @benchmark logl!($obs, $β, $L, $σ², true)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     7.126 μs (0.00% GC)
  median time:      7.278 μs (0.00% GC)
  mean time:        7.337 μs (0.00% GC)
  maximum time:     23.092 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     5

My median runt time is 2.1μs. You will get full credit (10 pts) if the median run time is within 10μs.

In [29]:
#  The points you will get are
clamp(10 / (median(bm_objgrad).time / 1e3) * 10, 0, 10)

10.0

## Q3. LmmModel type
We create a `LmmModel` type to hold all data points and model parameters. Log-likelihood/gradient of a LmmModel object is simply the sum of log-likelihood/gradient of individual data points.

In [5]:
# define a type that holds LMM model (data + parameters)
struct LmmModel{T <: AbstractFloat} <: MathProgBase.AbstractNLPEvaluator
    # data
    data :: Vector{LmmObs{T}}
    # parameters
    β    :: Vector{T}
    L    :: Matrix{T}
    σ²   :: Vector{T}    
    # arrays for holding gradient
    ∇β   :: Vector{T}
    ∇σ²  :: Vector{T}
    ∇L   :: Matrix{T}
    # TODO: add whatever intermediate arrays you may want to pre-allocate
    xty  :: Vector{T}
    ztr2 :: Vector{T}
    xtx  :: Matrix{T}
    ztz2 :: Matrix{T}
end

In [6]:

"""
    LmmModel(data::Vector{LmmObs})

Create an LMM model that contains data and parameters.
"""
function LmmModel(obsvec::Vector{LmmObs{T}}) where T <: AbstractFloat
    # dims
    p    = size(obsvec[1].X, 2)
    q    = size(obsvec[1].Z, 2)
    # parameters
    β    = Vector{T}(undef, p)
    L    = Matrix{T}(undef, q, q)
    σ²   = Vector{T}(undef, 1)    
    # gradients
    ∇β   = similar(β)    
    ∇σ²  = similar(σ²)
    ∇L   = similar(L)
    # intermediate arrays
    xty  = Vector{T}(undef, p)
    ztr2 = Vector{T}(undef, abs2(q))
    xtx  = Matrix{T}(undef, p, p)
    ztz2 = Matrix{T}(undef, abs2(q), abs2(q))
    LmmModel(obsvec, β, L, σ², ∇β, ∇σ², ∇L, xty, ztr2, xtx, ztz2)
end

"""
    logl!(m::LmmModel, needgrad=false)

Evaluate the log-likelihood of an LMM model at parameter values `m.β`, `m.L`, 
and `m.σ²`. If `needgrad==true`, then `m.∇β`, `m.∇Σ`, and `m.σ² are filled 
with the corresponding gradient.
"""
function logl!(m::LmmModel{T}, needgrad::Bool = false) where T <: AbstractFloat
    logl = zero(T)
    if needgrad
        fill!(m.∇β , 0)
        fill!(m.∇L , 0)
        fill!(m.∇σ², 0)        
    end
    @inbounds for i in 1:length(m.data)
        obs = m.data[i]
        logl += logl!(obs, m.β, m.L, m.σ²[1], needgrad)
        if needgrad
            BLAS.axpy!(T(1), obs.∇β, m.∇β)
            BLAS.axpy!(T(1), obs.∇Σ, m.∇L)
            m.∇σ²[1] += obs.∇σ²[1]
        end
    end
    # obtain gradient wrt L: m.∇L = m.∇L * L
    if needgrad
        # TODO 
        #m.∇L .= m.∇Σ * m.L
        BLAS.trmm!('R', 'L', 'N', 'N', T(1), m.L, m.∇L)
    end
    logl
end

logl!

## Q4. (20 pts) Test data
Let's generate a fake longitudinal data set to test our algorithm.

In [7]:
Random.seed!(257)

# dimension
m      = 1000 # number of individuals
ns     = rand(1500:2000, m) # numbers of observations per individual
p      = 5 # number of fixed effects, including intercept
q      = 3 # number of random effects, including intercept
obsvec = Vector{LmmObs{Float64}}(undef, m)
# true parameter values
βtrue  = [0.1; 6.5; -3.5; 1.0; 5]
σ²true = 1.5
σtrue  = sqrt(σ²true)
Σtrue  = Matrix(Diagonal([2.0; 1.2; 1.0]))
Ltrue  = Matrix(cholesky(Symmetric(Σtrue)).L)
# generate data
for i in 1:m
    # first column intercept, remaining entries iid std normal
    X = Matrix{Float64}(undef, ns[i], p)
    X[:, 1] .= 1
    @views Distributions.rand!(Normal(), X[:, 2:p])
    # first column intercept, remaining entries iid std normal
    Z = Matrix{Float64}(undef, ns[i], q)
    Z[:, 1] .= 1
    @views Distributions.rand!(Normal(), Z[:, 2:q])
    # generate y
    y = X * βtrue .+ Z * (Ltrue * randn(q)) .+ σtrue * randn(ns[i])
    # form a LmmObs instance
    obsvec[i] = LmmObs(y, X, Z)
end
# form a LmmModel instance
lmm = LmmModel(obsvec);

For later comparison with other software, we save the data into a text file lmm_data.txt. Do not put this file in Git. It takes 246.6MB storage.

In [8]:
(isfile("lmm_data.txt") && filesize("lmm_data.txt") == 246638945) || 
open("lmm_data.txt", "w") do io
    p = size(lmm.data[1].X, 2)
    q = size(lmm.data[1].Z, 2)
    # print header
    print(io, "ID,Y,")
    for j in 1:(p-1)
        print(io, "X" * string(j) * ",")
    end
    for j in 1:(q-1)
        print(io, "Z" * string(j) * (j < q-1 ? "," : "\n"))
    end
    # print data
    for i in eachindex(lmm.data)
        obs = lmm.data[i]
        for j in 1:length(obs.y)
            # id
            print(io, i, ",")
            # Y
            print(io, obs.y[j], ",")
            # X data
            for k in 2:p
                print(io, obs.X[j, k], ",")
            end
            # Z data
            for k in 2:q-1
                print(io, obs.Z[j, k], ",")
            end
            print(io, obs.Z[j, q], "\n")
        end
    end
end

## 4.1  Correctness
Evaluate log-likelihood and gradient of whole data set at the true parameter values.

In [30]:
copy!(lmm.β, βtrue)
copy!(lmm.L, Ltrue)
lmm.σ²[1] = σ²true
@show obj = logl!(lmm, true)
@show lmm.∇β
@show lmm.∇σ²
@show lmm.∇L;

obj = logl!(lmm, true) = -2.8547126486833044e6
lmm.∇β = [-8.69374436063768, 169.38364540289376, 1412.9462826020176, 583.9807190830627, 57.76586042026756]
lmm.∇σ² = [-371.82886426343083]
lmm.∇L = [20.19709774965016 -4.566719695066214 9.073934365208512; -5.895609775262915 -13.456093353754492 -18.889943728354076; 12.832481043361337 -20.692896580045577 -61.98953657916843]


Test correctness. You will loss all 20 points if following code throws AssertError.

In [168]:
@assert abs(obj - (-2.854712648683302e6)) < 1e-6
@assert norm(lmm.∇β - [-8.693744360651923, 169.38364540290684, 
        1412.9462826018173, 583.9807190830952, 57.76586042024306]) < 1e-6
@assert norm(lmm.∇L - [20.197097749713322 -4.566719695067792 9.073934365205824; 
        -5.895609775264991 -13.456093353707153 -18.889943728349024; 
        12.832481043357378 -20.69289658004 -61.98953657919662]) < 1e-6
@assert abs(lmm.∇σ²[1] - (-371.8288642639822)) < 1e-6

## 4.2  Efficiency
Test efficiency.

In [32]:
bm_model = @benchmark logl!($lmm, true)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.177 ms (0.00% GC)
  median time:      2.339 ms (0.00% GC)
  mean time:        2.535 ms (0.00% GC)
  maximum time:     73.461 ms (0.00% GC)
  --------------
  samples:          1969
  evals/sample:     1

My median run time is 2.122ms. You will get full credit if your median run time is within 10ms. The points you will get are

In [57]:
clamp(10 / (median(bm_model).time / 1e6) * 10, 0, 10)

10.0

## 4.3  Memory
You will lose 1 point for each 100 bytes memory allocation. So the points you will get is

In [56]:
clamp(10 - median(bm_model).memory / 100, 0, 10)

10.0

## 5  Q5. (30 pts) Starting point


$$
    \text{minimize} \sum_i \| \mathbf{r}_i^{(0)} \mathbf{r}_i^{(0)T} - \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T \|_{\text{F}}^2.
$$
Let $ \mathbf{r}_i^{(0)} \mathbf{r}_i^{(0)T} - \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T = \mathbf{A}_i$. Then we are minimizing $\sum_i tr(\mathbf{A}^T_i\mathbf{A}_i)$. We can minimize by setting its gradient to zero.
$$
\mathbf{D}_{\boldsymbol{\Sigma}} \mathbf{A}_i = \mathbf{Z}_i \otimes \mathbf{Z}_i \\
\mathbf{D}_{\mathbf{A}_i} \text{objective} = 2 \sum_i \mathbf{A}_i \\
\mathbf{D}_{\boldsymbol{\Sigma}} \text{objective} = \mathbf{D}_{\mathbf{A}_i} \text{objective} \cdot \mathbf{D}_{\boldsymbol{\Sigma}} \mathbf{A}_i \\
= 2 (vec(\mathbf{A}_i))^T (\mathbf{Z}_i \otimes \mathbf{Z}_i) \\
= 2 (\mathbf{Z}_i \otimes \mathbf{Z}_i)vec(\mathbf{A}_i) \\
= 2 vec\mathbf{Z}^T_i\mathbf{A}_i\mathbf{Z}_i = 2 \mathbf{Z}^T_i\mathbf{A}_i\mathbf{Z}_i \\
\text{set to}= 0
$$

$$
\implies \sum_i (\mathbf{Z}^T_i\mathbf{r}_i\mathbf{r}^T_i\mathbf{Z}_i - \mathbf{Z}^T_i\mathbf{Z}_i \boldsymbol{\Sigma^{(0)}} \mathbf{Z}^T_i\mathbf{Z}_i) = 0 \\
\implies \sum_i \mathbf{Z}^T_i\mathbf{r}_i\mathbf{r}^T_i\mathbf{Z}_i = \sum_i \mathbf{Z}^T_i\mathbf{Z}_i \boldsymbol{\Sigma^{(0)}} \mathbf{Z}^T_i\mathbf{Z}_i \\
\implies \sum_i vec(\mathbf{Z}^T_i\mathbf{r}_i\mathbf{r}^T_i\mathbf{Z}_i) = \sum_i vec(\mathbf{Z}^T_i\mathbf{Z}_i \boldsymbol{\Sigma^{(0)}} \mathbf{Z}^T_i\mathbf{Z}_i) \\
\implies (\sum_i\mathbf{Z}^T_i\mathbf{r}_i \otimes \mathbf{r}^T_i\mathbf{Z}_i) = (\sum_i\mathbf{Z}^T_i\mathbf{Z}_i \otimes \mathbf{Z}^T_i\mathbf{Z}_i)vec(\boldsymbol{\Sigma^{(0)}}) \\
\implies vec(\boldsymbol{\Sigma^{(0)}}) = (\sum_i\mathbf{Z}^T_i\mathbf{Z}_i \otimes \mathbf{Z}^T_i\mathbf{Z}_i)^{-1} (\sum_i \mathbf{Z}^T_i\mathbf{r}_i \otimes \mathbf{r}^T_i\mathbf{Z}_i)
$$

We implement this start point strategy in the function `init_ls()`.

In [11]:
"""
    init_ls!(m::LmmModel)

Initialize parameters of a `LmmModel` object from the least squares estimate. 
`m.β`, `m.L`, and `m.σ²` are overwritten with the least squares estimates.
"""
function init_ls!(m::LmmModel{T}) where T <: AbstractFloat
    p, q = size(m.data[1].X, 2), size(m.data[1].Z, 2)
    # TODO: fill m.β, m.L, m.σ² by LS estimates
    # get inv(sum(xtx)) and then m.β, and get m.L,
    fill!(m.β, 0)
    fill!(m.xtx, 0)
    fill!(m.xty, 0)
    fill!(m.L, 0)
    fill!(m.ztr2, 0)
    fill!(m.ztz2, 0)  
    m.σ²[1] = 0.0
    ns = 0
    @inbounds for i in 1:length(m.data)
        obs = m.data[i]
        BLAS.axpy!(T(1), obs.xtx, m.xtx)
        BLAS.axpy!(T(1), obs.xty, m.β)
        #BLAS.gemv!('N', T(2), obs.ztx, m.L, T(1), m.L)
    end
    LAPACK.potrf!('U', m.xtx)
    LAPACK.potrs!('U', m.xtx, m.β)
    
    # Get m.σ² and Σ
    @inbounds for i in 1:length(m.data)
        obs = m.data[i]
        m.σ²[1] += obs.yty
        BLAS.gemv!('N', T(1), obs.xtx, m.β, T(0), m.xty)
        m.σ²[1] += dot(m.β, m.xty)
        m.σ²[1] -= 2dot(m.β, obs.xty)
        ns += size(obs.X, 1)
        # get Σ
        # get ztxβ, then zty - ztxβ
        BLAS.gemv!('N', T(1), obs.ztx, m.β, T(0), obs.storage_q3)
        BLAS.axpby!(T(1), obs.zty, T(-1), obs.storage_q3)
        # get Kronecker product from lefthand side
        BLAS.gemm!('N', 'T', T(1), obs.storage_q3, obs.storage_q3, T(0), obs.storage_qq2)
        m.ztr2 .+= Base.vec(obs.storage_qq2)
        m.ztz2 .+= Base.kron(obs.ztz, obs.ztz)
    end
    m.σ²[1] = m.σ²[1] / ns
    LAPACK.potrf!('U', m.ztz2)
    LAPACK.potrs!('U', m.ztz2, m.ztr2)
    m.L .= reshape(m.ztr2, q, q)
    LAPACK.potrf!('U', m.L)
    
    #sleep(1e-3) # pretend this takes 1ms
    m
end

init_ls!

In [12]:
init_ls!(lmm)
@show logl!(lmm)
@show lmm.β
@show lmm.σ²
@show lmm.L;

logl!(lmm) = -3.375070980624222e6
lmm.β = [0.08757868042872421, 6.49874064773743, -3.4970528646610326, 0.9971817478195717, 5.000992653697893]
lmm.σ² = [5.66724466129435]
lmm.L = [1.4307910693942878 -0.0020939271680232513 0.013217721346714471; -0.002995972291969741 1.08951604771318 -0.011539720315817311; 0.018911797660621295 -0.012600387416032015 0.9605029322102014]


## 5.1  Correctness
Your start points should have a log-likelihood larger than -3.375071 (10 pts). The points you get are

In [13]:
# this is the points you get
(logl!(lmm) >  -3.375071e6) * 10

10

## 5.2  Efficiency
The start point should be computed quickly. Otherwise there is no point using it as a starting point. You get full credit (10 pts) if the median run time is within 1ms.

In [14]:
bm_init = @benchmark init_ls!($lmm)

BenchmarkTools.Trial: 
  memory estimate:  796.97 KiB
  allocs estimate:  3002
  --------------
  minimum time:     589.320 μs (0.00% GC)
  median time:      676.489 μs (0.00% GC)
  mean time:        804.261 μs (10.70% GC)
  maximum time:     13.486 ms (92.86% GC)
  --------------
  samples:          6194
  evals/sample:     1

In [15]:
# this is the points you get
clamp(1 / (median(bm_init).time / 1e6) * 10, 0, 10)

10.0

## 6  Q6. NLP via MathProgBase.jl
We define the NLP problem using the modelling tool MathProgBase.jl. Start-up code is given below. Modify if necessary to accomodate your own code.

In [16]:
"""
    fit!(m::LmmModel, solver=Ipopt.IpoptSolver(print_level=5))

Fit an `LmmModel` object by MLE using a nonlinear programming solver. Start point 
should be provided in `m.β`, `m.σ²`, `m.L`.
"""
function fit!(
        m::LmmModel,
        solver=Ipopt.IpoptSolver(print_level=5)
    )
    p    = size(m.data[1].X, 2)
    q    = size(m.data[1].Z, 2)
    npar = p + ((q * (q + 1)) >> 1) + 1
    optm = MathProgBase.NonlinearModel(solver)
    # set lower bounds and upper bounds of parameters
    # diagonal entries of Cholesky factor L should be >= 0
    lb   = fill(-Inf, npar)
    ub   = fill( Inf, npar)
    offset = p + 1
    for j in 1:q, i in j:q
        i == j && (lb[offset] = 0)
        offset += 1
    end
    # σ² should be >= 0
    lb[end] = 0
    MathProgBase.loadproblem!(optm, npar, 0, lb, ub, Float64[], Float64[], :Max, m)
    # starting point
    par0 = zeros(npar)
    modelpar_to_optimpar!(par0, m)
    MathProgBase.setwarmstart!(optm, par0)
    # optimize
    MathProgBase.optimize!(optm)
    optstat = MathProgBase.status(optm)
    optstat == :Optimal || @warn("Optimization unsuccesful; got $optstat")
    # update parameters and refresh gradient
    optimpar_to_modelpar!(m, MathProgBase.getsolution(optm))
    logl!(m, true)
    m
end

"""
    modelpar_to_optimpar!(par, m)

Translate model parameters in `m` to optimization variables in `par`.
"""
function modelpar_to_optimpar!(
        par :: Vector,
        m   :: LmmModel
    )
    p = size(m.data[1].X, 2)
    q = size(m.data[1].Z, 2)
    # β
    copyto!(par, m.β)
    # L
    offset = p + 1
    @inbounds for j in 1:q, i in j:q
        par[offset] = m.L[i, j]
        offset += 1
    end
    # σ²
    par[end] = m.σ²[1]
    par
end

"""
    optimpar_to_modelpar!(m, par)

Translate optimization variables in `par` to the model parameters in `m`.
"""
function optimpar_to_modelpar!(
        m   :: LmmModel, 
        par :: Vector
    )
    p = size(m.data[1].X, 2)
    q = size(m.data[1].Z, 2)
    # β
    copyto!(m.β, 1, par, 1, p)
    # L
    fill!(m.L, 0)
    offset = p + 1
    @inbounds for j in 1:q, i in j:q
        m.L[i, j] = par[offset]
        offset   += 1
    end
    # σ²
    m.σ²[1] = par[end]    
    m
end

function MathProgBase.initialize(
        m                  :: LmmModel, 
        requested_features :: Vector{Symbol}
    )
    for feat in requested_features
        if !(feat in [:Grad])
            error("Unsupported feature $feat")
        end
    end
end

MathProgBase.features_available(m::LmmModel) = [:Grad]

function MathProgBase.eval_f(
        m   :: LmmModel, 
        par :: Vector
    )
    optimpar_to_modelpar!(m, par)
    logl!(m, false) # don't need gradient here
end

function MathProgBase.eval_grad_f(
        m    :: LmmModel, 
        grad :: Vector, 
        par  :: Vector
    )
    p = size(m.data[1].X, 2)
    q = size(m.data[1].Z, 2)
    optimpar_to_modelpar!(m, par) 
    obj = logl!(m, true)
    # gradient wrt β
    copyto!(grad, m.∇β)
    # gradient wrt L
    offset = p + 1
    @inbounds for j in 1:q, i in j:q
        grad[offset] = m.∇L[i, j]
        offset += 1
    end
    # gradient with respect to σ²
    grad[end] = m.∇σ²[1]
    # return objective
    obj
end

MathProgBase.eval_g(m::LmmModel, g, par) = nothing
MathProgBase.jac_structure(m::LmmModel) = Int[], Int[]
MathProgBase.eval_jac_g(m::LmmModel, J, par) = nothing

## 7  Q7. (20 pts) Test drive
Now we can run our NLP solver to compute the MLE. For grading purpose, we first use the :LD_LBFGS (limited-memory BFGS) algorithm in NLopt.jl here.

In [17]:
# initialize from least squares
init_ls!(lmm)
println("objective value at starting point: ", logl!(lmm)); println()

@time fit!(lmm, NLopt.NLoptSolver(algorithm=:LD_LBFGS, 
        ftol_rel = 1e-12, ftol_abs = 1e-12, 
        xtol_rel = 1e-12, xtol_abs = 1e-12, 
        maxeval=10000));

println("objective value at solution: ", logl!(lmm)); println()
println("solution values:")
@show lmm.β
@show lmm.σ²
@show lmm.L * transpose(lmm.L)
println("gradient @ solution:")
@show lmm.∇β
@show lmm.∇σ²
@show lmm.∇L
@show sqrt(abs2(norm(lmm.∇β)) + abs2(norm(lmm.∇σ²) + 
        abs2(norm(LowerTriangular(lmm.∇L)))))

objective value at starting point: -3.375070980624222e6

  0.777064 seconds (1.58 M allocations: 73.614 MiB)
objective value at solution: -2.8547097852113894e6

solution values:
lmm.β = [0.08237464781054629, 6.500144678053184, -3.498791279668385, 1.0004995438209268, 5.000049494751214]
lmm.σ² = [1.4990428254359849]
lmm.L * transpose(lmm.L) = [2.0569561620228005 -0.010074595940203004 0.018527099248931592; -0.010074595940203004 1.1822481218686605 -0.02268192815382304; 0.018527099248931592 -0.02268192815382304 0.9379055458283312]
gradient @ solution:
lmm.∇β = [-0.025276005391447143, -0.004816389129828735, 0.016564279758565448, -0.0008153107416575267, 0.03471473220162835]
lmm.∇σ² = [-0.059482263596102314]
lmm.∇L = [-0.02816976625617268 0.0746616094448793 -0.024956777417154626; 0.09733211273999222 0.030356434621579678 -0.01885979732603346; -0.0368181557016089 -0.02120634330968573 0.001192501009617314]
sqrt(abs2(norm(lmm.∇β)) + abs2(norm(lmm.∇σ²) + abs2(norm(LowerTriangular(lmm.∇L))))) = 0.08

0.08599548547514116

### 7.1  Correctness
You get 10 points if the following code does not throw AssertError.

In [18]:
# objective at solution should be close enough to the optimal
@assert logl!(lmm) > -2.85471e6
# gradient at solution should be small enough
@assert sqrt(abs2(norm(lmm.∇β)) + abs2(norm(lmm.∇σ²) + 
        abs2(norm(LowerTriangular(lmm.∇L))))) < 0.15

### 7.2  Efficiency
My median run time is 132.5ms. You get 10 points if your median time is within 1s(=1000ms).

In [19]:
bm_bfgs = @benchmark fit!($lmm, $(NLopt.NLoptSolver(algorithm=:LD_LBFGS, 
        ftol_rel = 1e-12, ftol_abs = 1e-12, 
        xtol_rel = 1e-12, xtol_abs = 1e-12, 
        maxeval = 10000))) setup = (init_ls!(lmm))

BenchmarkTools.Trial: 
  memory estimate:  11.02 KiB
  allocs estimate:  237
  --------------
  minimum time:     137.567 ms (0.00% GC)
  median time:      144.505 ms (0.00% GC)
  mean time:        146.997 ms (0.00% GC)
  maximum time:     164.424 ms (0.00% GC)
  --------------
  samples:          34
  evals/sample:     1

In [20]:
# this is the points you get
clamp(1 / (median(bm_bfgs).time / 1e9) * 10, 0, 10)

10.0

## 8  Q8. (10 pts) Gradient free vs gradient-based methods
Advantage of using a modelling tool such as MathProgBase.jl is that we can easily switch the backend solvers. For a research problem, we never know beforehand which solvers work better than the others.
Try different solvers in the NLopt.jl and Ipopt.jl packages. Compare the results in terms run times (the shorter the better), objective values at solution (the larger the better), and gradients at solution (closer to 0 the better). Summarize what you find.
See this page for the descriptions of algorithms in NLopt.
Documentation for the Ipopt can be found here.

In [21]:
# vector of solvers to compare
solvers = [
    # NLopt: gradient-based algorithms
    NLopt.NLoptSolver(algorithm=:LD_LBFGS, 
        ftol_rel = 1e-12, ftol_abs = 1e-12,
        xtol_rel = 1e-12, xtol_abs = 1e-12,
        maxeval=10000),
    NLopt.NLoptSolver(algorithm=:LD_MMA, 
        ftol_rel = 1e-12, ftol_abs = 1e-12, 
        xtol_rel = 1e-12, xtol_abs = 1e-12, 
        maxeval=10000),
    # NLopt: gradient-free algorithms
    NLopt.NLoptSolver(algorithm=:LN_BOBYQA, 
        ftol_rel = 1e-12, ftol_abs = 1e-12, 
        xtol_rel = 1e-12, xtol_abs = 1e-12, 
        maxeval=10000),
    # Ipopt
    Ipopt.IpoptSolver(print_level=0)
]
# containers for results
runtime = zeros(length(solvers))
objvals = zeros(length(solvers))
gradnrm = zeros(length(solvers))

for (i, solver) in enumerate(solvers)
    bm = @benchmark fit!($lmm, $solver) setup = (init_ls!(lmm))
    runtime[i] = median(bm).time / 1e9
    objvals[i] = logl!(lmm, true)
    gradnrm[i] = sqrt(abs2(norm(lmm.∇β)) + abs2(norm(lmm.∇σ²) + 
        abs2(norm(LowerTriangular(lmm.∇L)))))
end


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************



In [22]:
DataFrame(Runtime = runtime, Objective = objvals, Gradnorm = gradnrm)

,Runtime,Objective,Gradnorm
,Float64,Float64,Float64
1,0.158832,-2.85471e6,0.0859955
2,0.161323,-2.85471e6,0.00449421
3,0.0604673,-2.85603e6,35203.7
4,5.37031,-2.85471e6,1.55889e-5


## 9  Q9. (10 pts) Compare with existing art
Let's compare our method with lme4 package in R and MixedModels.jl package in Julia. Both lme4 and MixedModels.jl are developed mainly by Doug Bates. Summarize what you find.

In [23]:
method  = ["257", "lme4", "MixedModels.jl"]
runtime = zeros(3)  # record the run times
loglike = zeros(3); # record the log-likelihood at MLE

### 9.1  Your approach

In [24]:
bm_257 = @benchmark fit!($lmm, $(NLopt.NLoptSolver(algorithm=:LD_MMA, 
        ftol_rel = 1e-12, ftol_abs = 1e-12, 
        xtol_rel = 1e-12, xtol_abs = 1e-12, 
        maxeval=10000))) setup=(init_ls!(lmm))
runtime[1] = (median(bm_257).time) / 1e9
loglike[1] = logl!(lmm)

-2.854709785203477e6

### 9.2  lme4

In [25]:
R"""
library(lme4)
library(readr)
library(magrittr)

testdata <- read_csv("lmm_data.txt")
"""

┌ Warning: RCall.jl: Loading required package: Matrix
└ @ RCall /Users/caesar/.julia/packages/RCall/Qzssx/src/io.jl:160
┌ Warning: RCall.jl: Parsed with column specification:
│ cols(
│   ID = col_double(),
│   Y = col_double(),
│   X1 = col_double(),
│   X2 = col_double(),
│   X3 = col_double(),
│   X4 = col_double(),
│   Z1 = col_double(),
│   Z2 = col_double()
│ )
└ @ RCall /Users/caesar/.julia/packages/RCall/Qzssx/src/io.jl:160


RObject{VecSxp}
# A tibble: 1,753,910 x 8
      ID      Y      X1      X2     X3      X4      Z1     Z2
   <dbl>  <dbl>   <dbl>   <dbl>  <dbl>   <dbl>   <dbl>  <dbl>
 1     1   2.30  2.16    1.71    1.95  -0.744   0.441  -0.155
 2     1  -7.82  0.591   0.0736 -1.39  -1.53   -0.284  -0.283
 3     1  -1.05 -0.666   0.223  -1.07   0.726  -0.701  -0.639
 4     1  -5.42 -0.364   0.181   1.61  -0.623   0.0332  0.905
 5     1  -4.23  0.880   1.91   -0.999 -0.371   1.03    0.581
 6     1  -2.72  0.341   0.162   0.603 -0.285   1.42   -0.831
 7     1   1.13 -0.0213 -0.500   0.178  0.747   0.638  -0.617
 8     1 -17.6  -1.76    0.899  -0.931 -0.0527  0.353  -1.38 
 9     1  -4.30 -0.207   0.248  -0.362 -0.0995 -0.587   0.266
10     1  20.9   2.24    0.523  -1.47   2.07    0.714   2.06 
# … with 1,753,900 more rows


In [26]:
R"""
rtime <- system.time(mmod <- 
  lmer(Y ~ X1 + X2 + X3 + X4 + (1 + Z1 + Z2 | ID), testdata, REML = FALSE))
"""

RObject{RealSxp}
   user  system elapsed 
103.371   8.921 112.949 


In [27]:
R"""
rtime <- rtime["elapsed"]
summary(mmod)
rlogl <- logLik(mmod)
"""
runtime[2] = @rget rtime
loglike[2] = @rget rlogl;

### 9.3  MixedModels.jl

In [28]:
testdata = CSV.File("lmm_data.txt", types = Dict(1=>String)) |> DataFrame!

,ID,Y,X1,X2,X3,X4,Z1
,String,Float64,Float64,Float64,Float64,Float64,Float64
1,1,2.29562,2.16439,1.71386,1.94501,-0.743783,0.441018
2,1,-7.82426,0.591156,0.0735725,-1.38799,-1.53388,-0.28432
3,1,-1.05162,-0.665533,0.222603,-1.06705,0.726237,-0.700676
4,1,-5.41677,-0.363982,0.181063,1.6054,-0.623457,0.0332452
5,1,-4.22634,0.880121,1.91223,-0.998628,-0.370723,1.02984
6,1,-2.72221,0.341423,0.161899,0.602739,-0.285159,1.4209
7,1,1.13149,-0.0212729,-0.500254,0.177658,0.747107,0.638381
8,1,-17.6405,-1.76199,0.898998,-0.931399,-0.0526501,0.352804
9,1,-4.30473,-0.206732,0.248303,-0.362355,-0.0994689,-0.58678


In [29]:
mj = fit(MixedModel, @formula(Y ~ X1 + X2 + X3 + X4 + (1 + Z1 + Z2 | ID)), testdata)
bm_mm = @benchmark fit(MixedModel, @formula(Y ~ X1 + X2 + X3 + X4 + (1 + Z1 + Z2 | ID)), testdata)
loglike[3] = loglikelihood(mj)
runtime[3] = median(bm_mm).time / 1e9

1.2465281625

In [30]:
display(bm_mm)
mj

BenchmarkTools.Trial: 
  memory estimate:  920.42 MiB
  allocs estimate:  9433918
  --------------
  minimum time:     1.181 s (8.34% GC)
  median time:      1.247 s (15.48% GC)
  mean time:        1.272 s (15.19% GC)
  maximum time:     1.415 s (20.39% GC)
  --------------
  samples:          4
  evals/sample:     1

Linear mixed model fit by maximum likelihood
 Y ~ 1 + X1 + X2 + X3 + X4 + (1 + Z1 + Z2 | ID)
     logLik        -2 logLik          AIC             BIC       
 -2.85470979×10⁶  5.70941957×10⁶  5.70944357×10⁶   5.7095921×10⁶

Variance components:
            Column    Variance  Std.Dev.   Corr.
ID       (Intercept)  2.0568343 1.4341668
         Z1           1.1823375 1.0873534 -0.01
         Z2           0.9379257 0.9684656  0.01 -0.02
Residual              1.4990426 1.2243540
 Number of obs: 1753910; levels of grouping factors: 1000

  Fixed-effects parameters:
──────────────────────────────────────────────────────────
               Estimate    Std.Error      z value  P(>|z|)
──────────────────────────────────────────────────────────
(Intercept)   0.0823213  0.045357         1.81496   0.0695
X1            6.50014    0.000924999   7027.19      <1e-99
X2           -3.49879    0.000924895  -3782.91      <1e-99
X3            1.0005     0.000925797   1080.69      <1e-99
X4            5.0000

### 9.4  Summary

In [31]:
DataFrame(method = method, runtime = runtime, logl = loglike)

,method,runtime,logl
,String,Float64,Float64
1,257,0.156089,-2.85471e6
2,lme4,112.949,-2.85471e6
3,MixedModels.jl,1.24653,-2.85471e6


## My Summary

First of all, the Julia implementation is almost 100 folds faster than the R linear mixed model package lme4, as shown above in the table (given my machine's environment and conditions). From personal experience I can confirm lme4, as fast as it is compared to other R packages like nlme, it is still quite slow. Julia's MixedModels.jl finishing at around 1.25 seconds is impressive. However, due to the fact that we can further optimize it in this special case, our code gave us another almost 10 fold improvement. Specialization would of course bring in further optimization, at the expense of generality. Thus, it is nice that Julia's linear mixed model package, without the loss of generality, finds a sweet spot and optimize computations to almost as fast as C. Even though not general enough to replace MixedModels.jl, our implementation taught us how to optimize our code for any specific research project. It shows us that using a faster solfware alone is not enough to achieve the perfect optimization. Optimized softwares like Julia provides us with a platform and tools to implement our ideas efficiently. It is up to the user to use these tools to their fullest extent.

## 10  Q10. Be proud of yourself
Go to your resume/cv and claim you have experience performing analysis on complex longitudinal data sets with millions of records. And you beat current software by XXX fold.     

Answer: I definitely am. That was a biiig hw, but I'm glad I finally did it. Really with 257 was broken up into 2 or 3 classes so we can learn everything in depth instead of cramming through.